In this notebook, we provide an example (Diving48) on how to prepare 2D skeleton data for custom video datasets.
You can run each step in this notebook or implement them on your own. 

**Step 1**: Download Videos and Annotations from the official website of Diving48: http://www.svcl.ucsd.edu/projects/resound/dataset.html.

In [ ]:
# Download Videos and Annotations
!wget http://www.svcl.ucsd.edu/projects/resound/Diving48_V2_train.json
!wget http://www.svcl.ucsd.edu/projects/resound/Diving48_V2_test.json
!wget http://www.svcl.ucsd.edu/projects/resound/Diving48_rgb.tar.gz
# Extract the videos
!tar -xf Diving48_rgb.tar.gz
# After that, the folder structure will looks like:
# WorkingDirectory
# ├── Diving48_V2_train.json
# ├── Diving48_V2_test.json
# `-- rgb
#     ├-- _8Vy3dlHg2w_00000.mp4
#     ├-- _8Vy3dlHg2w_00001.mp4
#     ├-- _8Vy3dlHg2w_00004.mp4
#     ├-- ...

In [ ]:
# After extracting videos successfully, you can now remove the tar.gz file
!rm Diving48_rgb.tar.gz

**Step 2**: Generate video list for 2d skeleton extraction

In [3]:
from mmcv import load, dump
from pyskl.smp import *
train = load('Diving48_V2_train.json')
test = load('Diving48_V2_test.json')
tmpl = 'examples/extract_diving48_skeleton/rgb/{}.mp4'

lines = [(tmpl + ' {}').format(x['vid_name'], x['label']) for x in train + test]
mwlines(lines, 'diving48.list')

**Step 3**: Extract 2D skeleton for Diving48 Videos

In [4]:
os.chdir('../..')
# Extract 2D skeletons of diving48 videos with 8 GPUs (it may take around 12 hours). The video_list is diving48.list  the output file is diving48.pkl
!bash tools/dist_run.sh tools/data/custom_2d_skeleton.py 8 --video-list examples/extract_diving48_skeleton/diving48.list --out examples/extract_diving48_skeleton/diving48_annos.pkl

**Step 4**: Merge extracted 2D skeletons and split information to get the final annotation file

In [ ]:
os.chdir('examples/extract_diving48_skeleton')
train = load('Diving48_V2_train.json')
test = load('Diving48_V2_test.json')
annotations = load('diving48_annos.pkl')
split = dict()
split['train'] = [x['vid_name'] for x in train]
split['test'] = [x['vid_name'] for x in test]
dump(dict(split=split, annotations=annotations), 'diving48_hrnet.pkl')

Now you get the 2D skeleton annotations for diving48. We also provide the generated pickle file at https://download.openmmlab.com/mmaction/pyskl/data/diving48/diving48_hrnet.pkl. You can download this pickle file to check if it's the same as your generated pickle file. We also provide a [config file](https://github.com/kennymckormick/pyskl/blob/main/configs/posec3d/slowonly_r50_gym/joint.py) for training on diving48 and release the trained weights and accuracy number in [PoseC3D](https://github.com/kennymckormick/pyskl/blob/main/configs/posec3d/)